<a href="https://colab.research.google.com/github/Anjasfedo/Code-as-a-Cryptography/blob/main/eceg_koblitz_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import sympy as sp
import json
import base64

class EllipticCurveElGamal:
  def __init__(self, a, b, p, k, B=None):
    self.p = p # primer number
    self.a = a # alpha
    self.b = b # beta

    self.k = k
    self.PointB = B

    self.pointP = None
    self.pointQ = None


    self.char_to_num_dict = {
    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
    'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17,
    'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25,
    'q': 26, 'r': 27, 's': 28, 't': 29, 'u': 30, 'v': 31, 'w': 32, 'x': 33,
    'y': 34, 'z': 35, '.': 36, '#': 37, '*': 38, '/': 39, '-': 40
    }

    self.num_to_char_dict = {v: k for k, v in self.char_to_num_dict.items()}

    self.k_koblitz = max(self.char_to_num_dict.values()) + 1  # max 24 for 35 char

  def example_function(self):
      print(f"Using persistent k value: {self.k}")

  def elliptic_curve_equation(self, x):
    return (x**3 + self.a*x + self.b) % self.p

  def is_on_curve(self, x, y):
    return self.elliptic_curve_equation(x)  == (y**2) % p

  # def generate_random_number(self):
  #   self.k = random.randint(1, self.p - 1)
  #   return self.k

  def generate_random_point(self):
    while True:
      x = random.randint(1, self.p - 1)
      y = random.randint(1, self.p - 1)
      if self.is_on_curve(x, y):
        return (x, y)

  def mod_inverse(self, a, p):
      if a == 0:
          raise ValueError("Inverse does not exist for 0.")
      return pow(a, p - 2, p)

  def calculate_slope_mod(self, P, Q):
    x1, y1 = P
    x2, y2 = Q
    if x1 == x2 and y1 == y2:
        # Use the formula for point doubling when P == Q
        numerator = (3 * x1**2 + self.a) % self.p
        denominator = (2 * y1) % self.p
    else:
        # Use the formula for regular slope when P != Q
        numerator = (y2 - y1) % self.p
        denominator = (x2 - x1) % self.p

    if denominator == 0:
        raise ValueError("Slope is undefined (denominator is zero).")

    # Compute the slope as (numerator / denominator) % p, which is
    # numerator * mod_inverse(denominator, p) % p
    slope = (numerator * self.mod_inverse(denominator, self.p)) % self.p
    return slope

  def calculate_add_xr_mod(self, P, Q, m):
    x1, y1 = P
    x2, y2 = Q
    xr = (m**2 - x1 - x2) % self.p
    return xr

  def calculate_add_yr_mod(self, P, Q, m, xr):
    x1, y1 = P
    x2, y2 = Q
    yr = (m * (x1 - xr) - y1) % self.p
    return yr

  def calculate_point_addition(self, P, Q):
    m = self.calculate_slope_mod(P, Q)

    xr = self.calculate_add_xr_mod(P, Q, m)

    yr = self.calculate_add_yr_mod(P, Q, m, xr)

    R = (xr, yr)

    return R

  def calculate_dob_xr_mod(self, P, Q, m):
    x1, y1 = P
    x2, y2 = Q
    xr = (m**2 - (2 * x1)) % self.p
    return xr

  def calculate_dob_yr_mod(self, P, Q, m, xr):
    x1, y1 = P
    yr = (m * (x1 - xr) - y1) % self.p
    return yr

  def calculate_point_doubling(self, P):

    m = self.calculate_slope_mod(P, P)

    xr = self.calculate_dob_xr_mod(P, P, m)

    yr = self.calculate_dob_yr_mod(P, P, m, xr)

    R = (xr, yr)

    return R

  def calculate_point_multiplication(self, P, k):
    if k == 0:
      return None
    elif k == 1:
      return P

    # Initialize R to be the point at infinity, often represented as None in this context
    R = None
    Q = P  # Start with Q as P

    # Double-and-add method
    while k > 0:
        if k % 2 == 1:  # If k is odd, add Q to the result
            if R is None:
                R = Q  # R is the point at infinity initially
            else:
                R = self.calculate_point_addition(R, Q)
        Q = self.calculate_point_addition(Q, Q)  # Double the point Q
        k //= 2  # Move to the next bit

    return R

  def calculate_point_subtract(self, P, Q):
    # Find the inverse of point Q (x_Q, y_Q) -> (x_Q, -y_Q mod p)
    Q_inv = (Q[0], (-Q[1]) % self.p)

    # Subtract P - Q by adding P and Q_inv
    R = self.calculate_point_addition(P, Q_inv)

    return R

  def generate_public_key(self, B, private_key):
    public_key = self.calculate_point_multiplication(B, private_key)
    return public_key

  def encryption(self, M, public_key):
      max_attempts = 100  # Limit the number of retries for generating valid `k`
      attempts = 0

      while attempts < max_attempts:
          k = random.randint(1, self.p - 1)  # Generate random `k`, 1 < k < p - 1

          # Calculate C1 = k * B (public generator point)
          C1 = self.calculate_point_multiplication(self.PointB, k)

          # If C1 is the point at infinity (None), retry with a new `k`
          if C1 is None:
              attempts += 1
              continue

          # Calculate C2 = M + k * public_key
          k_times_public_key = self.calculate_point_multiplication(public_key, k)
          if k_times_public_key is None:
              attempts += 1
              continue

          C2 = self.calculate_point_addition(M, k_times_public_key)

          # If C2 is the point at infinity (None), retry with a new `k`
          if C2 is None:
              attempts += 1
              continue

          # If both C1 and C2 are valid, return the encrypted pair (C1, C2)
          return (C1, C2)

      # If no valid (C1, C2) pair is found after max_attempts, raise an error
      raise ValueError(f"Failed to find valid encryption after {max_attempts} attempts")


  def decryption(self, C, private_key):
    C1, C2 = C

    _p = self.calculate_point_multiplication(C1, private_key)

    plain = self.calculate_point_subtract(C2, _p)

    return plain

  # Function to map char to num
  def char_to_num(self, char):
      # Ensure the input is a string; if not, convert it
      char = str(char)

      # Check if the string exists in the dictionary
      if char in self.char_to_num_dict:
          return self.char_to_num_dict[char]
      else:
          # Handle the expanded character set (0-40)
          raise ValueError(f"Character '{char}' is not valid. Please use '0-9', 'a-z', or other valid characters.")


  # Function to map num to char
  def num_to_char(self, num):
      # Ensure that the input is an integer; if it's a string digit, convert to integer
      if isinstance(num, str) and num.isdigit():
          num = int(num)

      # Now we expect num to be an integer
      if isinstance(num, int):
          if num in self.num_to_char_dict:  # Handle numbers 0-40 by converting them using the dictionary
              return self.num_to_char_dict[num]
          else:
              raise ValueError(f"Number '{num}' is out of the valid range (0-40).")
      else:
          raise ValueError(f"Input '{num}' is not valid. Please provide a valid number.")


  def message_to_koblitz(self, message):
      # Convert the message to a list of characters
      chars = list(message)

      koblitz = []
      for char in chars:
          num = self.char_to_num(char)
          koblitz.append(num)

      return koblitz

  def koblitz_to_message(self, koblitz):
      message = []
      for num in koblitz:
          char = self.num_to_char(num)
          message.append(char)

      return ''.join(message)

  def koblitz_encode(self, m, max_attempts=1000):
    num = 1  # Start with x = m * k + 1
    attempts = 0

    while attempts < max_attempts:
        x = m * self.k_koblitz + num
        rhs = (x**3 + self.a * x + self.b) % self.p  # right-hand side of the elliptic curve equation

        # Check if rhs is a quadratic residue modulo p
        if sp.is_quad_residue(rhs, self.p):
            y = sp.sqrt_mod(rhs, self.p)
            return (x, y)  # Return the point (x, y) as a tuple

        num += 1  # Increment to check next x value
        attempts += 1

    # If no valid point is found after max_attempts
    raise ValueError(f"No valid point found after {max_attempts} attempts for message {m}.")

  def koblitz_encode_message(self, message):
    encoded_points = []
    for char in message:
      encoded_point = self.koblitz_encode(char)
      encoded_points.append(encoded_point)
      # print(f"Encoded point for character '{char}': {encoded_point}")

    return encoded_points

  def koblitz_decode(self, x):
    return (x - 1) // self.k_koblitz

  def koblitz_decode_message(self, points):
      decoded_message = []
      for point in points:
          x, y = point  # You can access both x and y here
          decoded_num = self.koblitz_decode(x)  # Decode based on x-coordinate
          decoded_message.append(decoded_num)
      return decoded_message

  def encrypt_message(self, message, public_key):
      # Encode the message into Koblitz points
      koblitz_message = self.message_to_koblitz(message)
      encoded_message = self.koblitz_encode_message(koblitz_message)
      encrypted_points = []

      # Encrypt each Koblitz point
      for point in encoded_message:
          encrypted_point = self.encryption(point, public_key)
          encrypted_points.append(encrypted_point)
      print(encrypted_points)
      # Convert encrypted_points (a list of tuples) to a JSON string
      encrypted_message_string = json.dumps(encrypted_points)

      # Optionally, encode the string to base64 for a cleaner ciphertext
      encrypted_message_base64 = base64.b64encode(encrypted_message_string.encode()).decode()

      return encrypted_message_base64, encrypted_points  # Return as readable string

  def decrypt_message(self, encrypted_message_base64, private_key):
      # Decode the base64 string back to the JSON string
      encrypted_message_string = base64.b64decode(encrypted_message_base64).decode()

      # Convert the JSON string back to a list of encrypted points (tuples)
      encrypted_points = json.loads(encrypted_message_string)

      decrypted_message = []

      # Decrypt each point
      for point in encrypted_points:
          decrypted_point = self.decryption(point, private_key)
          decrypted_message.append(decrypted_point)

      # Decode the Koblitz points into the original numeric values
      decoded_koblitz_points = self.koblitz_decode_message(decrypted_message)

      # Convert numeric Koblitz points to characters using num_to_char
      plain_text_message = ''.join([self.num_to_char(num) for num in decoded_koblitz_points])

      return plain_text_message

In [ ]:
elliptic = EllipticCurveElGamal(a=1, b=1, p=1019, k=3, B=(2, 3))

message = "hello"
private_key = 5
public_key = elliptic.generate_public_key(B=(2, 3), private_key=private_key)

# Encrypt the message
encrypted_message_string, encrypted_points = elliptic.encrypt_message(message, public_key)
print(f"Encrypted message: {encrypted_message_string}")
print(f"Encrypted points: {encrypted_points}")

# Decrypt the message
decrypted_message = elliptic.decrypt_message(encrypted_message_string, private_key)
print(f"Decrypted message: {decrypted_message}")

[((130, 409), (250, 397)), ((813, 904), (523, 358)), ((29, 571), (320, 738)), ((349, 419), (37, 640)), ((441, 403), (403, 102))]
Encrypted message: W1tbMTMwLCA0MDldLCBbMjUwLCAzOTddXSwgW1s4MTMsIDkwNF0sIFs1MjMsIDM1OF1dLCBbWzI5LCA1NzFdLCBbMzIwLCA3MzhdXSwgW1szNDksIDQxOV0sIFszNywgNjQwXV0sIFtbNDQxLCA0MDNdLCBbNDAzLCAxMDJdXV0=
Encrypted points: [((130, 409), (250, 397)), ((813, 904), (523, 358)), ((29, 571), (320, 738)), ((349, 419), (37, 640)), ((441, 403), (403, 102))]
Decrypted message: hello


In [ ]:
# Function to flatten the list down to one level
def flatten_one_level(nested_list):
    return [item for sublist in nested_list for item in sublist]

# Function to revert the list back to the original structure
def unflatten_one_level(flat_list):
    return [(flat_list[i], flat_list[i + 1]) for i in range(0, len(flat_list), 2)]


In [ ]:
flat_points = flatten_one_level(encrypted_points)
unflat_points = unflatten_one_level(flat_points)

In [ ]:
# Function to encode (x, y) into a single character, ensuring it fits within Unicode limits
def koblitz_encode_to_char(x, y, field_size_bits=10, unicode_limit=0x110000):
    """
    Encodes the point (x, y) into a single integer and converts it to a character.
    The range of x and y is limited by the size of field_size_bits.

    :param x: x-coordinate of the point (must fit within the range of field_size_bits)
    :param y: y-coordinate of the point (must fit within the range of field_size_bits)
    :param field_size_bits: the bit length used for each coordinate, default is 10 bits
    :param unicode_limit: maximum Unicode range (0x110000)
    :return: a single character representing the encoded point
    """
    # Encode the point into a single integer (x in higher bits, y in lower bits)
    encoded_int = (x << field_size_bits) + y

    # Ensure the encoded integer fits within the Unicode range
    if encoded_int >= unicode_limit:
        raise ValueError("Encoded integer exceeds Unicode character range")

    # Convert integer to a Unicode character
    return chr(encoded_int)

# Function to decode from the character back to (x, y)
def koblitz_decode_from_char(encoded_char, field_size_bits=10):
    """
    Decodes the character back into an integer and extracts (x, y) coordinates.

    :param encoded_char: the encoded character
    :param field_size_bits: the bit length used for each coordinate, default is 10 bits
    :return: tuple (x, y) representing the point
    """
    # Convert character back to integer
    encoded_int = ord(encoded_char)

    # Extract x and y from the integer
    x = encoded_int >> field_size_bits
    y = encoded_int & ((1 << field_size_bits) - 1)
    return x, y

In [ ]:
kob1 = koblitz_encode_to_char(flat_points[0][0], flat_points[0][1])
kob1

'\U0003dc5d'

In [ ]:
print(kob1)

𽱝


In [ ]:
flat_points[0][0], flat_points[0][1]

(247, 93)

In [ ]:
kob2 = koblitz_decode_from_char(kob1)
kob2

(247, 93)

In [ ]:
def koblitz_encode_to_two_chars(x, y, field_size_bits=16, unicode_limit=0x10FFFF):
    """
    Encodes the point (x, y) into two Unicode characters.

    :param x: x-coordinate of the point
    :param y: y-coordinate of the point
    :param field_size_bits: number of bits for each coordinate, default is 16 bits
    :param unicode_limit: maximum Unicode range, default is 0x10FFFF
    :return: a tuple of two characters representing the encoded point
    """
    # Encode the point into a single large integer
    encoded_int = (x << field_size_bits) + y

    # Split the encoded integer into two parts
    high_char = chr(encoded_int >> 16)
    low_char = chr(encoded_int & 0xFFFF)

    return high_char, low_char

def koblitz_decode_from_two_chars(high_char, low_char, field_size_bits=16):
    """
    Decodes two characters back into an (x, y) point.

    :param high_char: the first character
    :param low_char: the second character
    :param field_size_bits: number of bits for each coordinate, default is 16 bits
    :return: tuple (x, y) representing the point
    """
    # Convert the two characters back into an integer
    encoded_int = (ord(high_char) << 16) + ord(low_char)

    # Extract x and y from the integer
    x = encoded_int >> field_size_bits
    y = encoded_int & ((1 << field_size_bits) - 1)

    return x, y

# Test with larger values
x, y = 50000, 60000
high_char, low_char = koblitz_encode_to_two_chars(x, y)
decoded_x, decoded_y = koblitz_decode_from_two_chars(high_char, low_char)

(high_char, low_char), (decoded_x, decoded_y)


(('썐', '\uea60'), (50000, 60000))

In [ ]:
char_to_num_dict = {
    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
    'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17,
    'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25,
    'q': 26, 'r': 27, 's': 28, 't': 29, 'u': 30, 'v': 31, 'w': 32, 'x': 33,
    'y': 34, 'z': 35, '.': 36, '#': 37, '*': 38, '/': 39, '-': 40
    }

In [ ]:
# Function to encode a string into a sequence of Unicode characters
def encode_string_to_unicode(input_str, field_size_bits=10, unicode_limit=0x110000):
    encoded_chars = []

    for i in range(0, len(input_str), 2):
        x = char_to_num_dict[input_str[i]]
        y = char_to_num_dict[input_str[i+1]] if i+1 < len(input_str) else 0  # Handle odd length

        encoded_char = koblitz_encode_to_char(x, y, field_size_bits, unicode_limit)
        encoded_chars.append(encoded_char)

    return ''.join(encoded_chars)

# Function to decode a sequence of Unicode characters back into a string
def decode_unicode_to_string(encoded_str, field_size_bits=10):
    num_to_char_dict = {v: k for k, v in char_to_num_dict.items()}  # Reverse the dictionary
    decoded_str = []

    for encoded_char in encoded_str:
        x, y = koblitz_decode_from_char(encoded_char, field_size_bits)
        decoded_str.append(num_to_char_dict[x])
        if y != 0:  # If y is not zero, decode the second character
            decoded_str.append(num_to_char_dict[y])

    return ''.join(decoded_str)

# Example usage
input_string = "hello"
encoded_str = encode_string_to_unicode(input_string)
decoded_str = decode_unicode_to_string(encoded_str)

encoded_str, decoded_str


('䐎吕怀', 'hello')

In [ ]:
# Function to encode a list of (x, y) points into a string of Unicode characters
def encode_points_to_unicode(points, field_size_bits=10, unicode_limit=0x110000):
    encoded_chars = []

    for x, y in points:
        encoded_char = koblitz_encode_to_char(x, y, field_size_bits, unicode_limit)
        encoded_chars.append(encoded_char)

    return ''.join(encoded_chars)

# Function to decode a string of Unicode characters back into a list of (x, y) points
def decode_unicode_to_points(encoded_str, field_size_bits=10):
    decoded_points = []

    for encoded_char in encoded_str:
        x, y = koblitz_decode_from_char(encoded_char, field_size_bits)
        decoded_points.append((x, y))

    return decoded_points

# Example usage with a list of points
points = flat_points
encoded_str = encode_points_to_unicode(points)
decoded_points = decode_unicode_to_points(encoded_str)

encoded_str, decoded_points


('\U0003dc5d덺痀\U000bf8c5\U000cb788\U00041e09\U00066f99\U000e5290ࠃ\U000b9c5b',
 [(247, 93),
  (44, 890),
  (29, 448),
  (766, 197),
  (813, 904),
  (263, 521),
  (411, 921),
  (916, 656),
  (2, 3),
  (743, 91)])

In [ ]:
flat_points

[(247, 93),
 (44, 890),
 (29, 448),
 (766, 197),
 (813, 904),
 (263, 521),
 (411, 921),
 (916, 656),
 (2, 3),
 (743, 91)]

In [ ]:
type(encoded_str)

str

In [ ]:
test_points = [((627, 692), (170, 274)), ((538, 674), (447, 303)), ((581, 20), (144, 561)), ((729, 78), (331, 367)), ((639, 408), (333, 316)), ((254, 504), (274, 422)), ((742, 74), (299, 568)), ((543, 61), (663, 494)), ((380, 155), (19, 138)), ((470, 441), (623, 47)), ((258, 276), (470, 441)), ((717, 601), (589, 672)), ((196, 638), (529, 497)), ((62, 590), (589, 672)), ((69, 21), (234, 563)), ((570, 511), (30, 515)), ((204, 363), (225, 117)), ((285, 96), (153, 436)), ((378, 579), (272, 76)), ((191, 29), (666, 666)), ((30, 515), (531, 557)), ((359, 455), (566, 344)), ((629, 206), (41, 274)), ((196, 638), (52, 346)), ((302, 291), (54, 620)), ((62, 590), (203, 169)), ((320, 577), (43, 429)), ((488, 692), (39, 402)), ((743, 597), (231, 575)), ((233, 495), (154, 575)), ((409, 470), (487, 380)), ((473, 515), (26, 312)), ((32, 451), (721, 510)), ((283, 54), (508, 668)), ((173, 33), (153, 436)), ((200, 220), (426, 201)), ((144, 561), (676, 558)), ((179, 494), (246, 698)), ((480, 495), (254, 504)), ((268, 522), (189, 728)), ((446, 219), (686, 393)), ((538, 77), (158, 722)), ((581, 731), (30, 515)), ((381, 69), (12, 516)), ((430, 102), (695, 581)), ((17, 419), (575, 329)), ((84, 138), (500, 383)), ((231, 575), (717, 601)), ((742, 677), (490, 544)), ((86, 184), (152, 414)), ((174, 240), (470, 441)), ((620, 645), (566, 344)), ((28, 449), (202, 738)), ((333, 316), (201, 5)), ((661, 572), (389, 146)), ((540, 477), (639, 343)), ((380, 155), (12, 516)), ((248, 236), (191, 722)), ((147, 576), (546, 399)), ((339, 353), (395, 141)), ((392, 341), (39, 402)), ((487, 380), (21, 618)), ((701, 203), (356, 696)), ((609, 626), (36, 336)), ((19, 138), (189, 23)), ((432, 441), (361, 743)), ((317, 181), (406, 194)), ((664, 108), (301, 493)), ((324, 7), (440, 457)), ((629, 545), (312, 95)), ((324, 744), (320, 577)), ((623, 47), (21, 133)), ((582, 15), (531, 557)), ((182, 84), (696, 661)), ((2, 373), (180, 343)), ((729, 673), (354, 598)), ((358, 551), (211, 731)), ((385, 423), (126, 476)), ((19, 138), (172, 496)), ((605, 454), (657, 156)), ((266, 507), (41, 477)), ((285, 655), (394, 324)), ((661, 179), (3, 540)), ((634, 657), (245, 509)), ((172, 255), (135, 198)), ((604, 541), (597, 328)), ((201, 5), (650, 58)), ((663, 257), (191, 722)), ((268, 229), (385, 328)), ((246, 698), (126, 275)), ((400, 513), (582, 15)), ((582, 736), (667, 571)), ((128, 499), (492, 584)), ((227, 677), (680, 657)), ((266, 244), (195, 694)), ((157, 696), (446, 532)), ((621, 596), (715, 83)), ((280, 504), (409, 470)), ((639, 343), (585, 643)), ((677, 419), (705, 112)), ((314, 715), (101, 747)), ((333, 435), (407, 553)), ((632, 80), (620, 106)), ((543, 61), (179, 494)), ((324, 7), (177, 164)), ((683, 343), (352, 65)), ((370, 740), (407, 198)), ((607, 733), (254, 247)), ((318, 672), (348, 228)), ((639, 343), (367, 629)), ((664, 643), (430, 102)), ((191, 29), (454, 471)), ((63, 62), (19, 613)), ((440, 294), (197, 644)), ((583, 177), (2, 373)), ((57, 419), (472, 336)), ((402, 29), (461, 92)), ((370, 740), (466, 608)), ((258, 276), (612, 588)), ((466, 608), (594, 141)), ((205, 590), (238, 55)), ((97, 284), (343, 546)), ((301, 493), (372, 714)), ((683, 408), (66, 537)), ((5, 526), (161, 1)), ((401, 598), (635, 509)), ((45, 654), (696, 661)), ((600, 310), (735, 705)), ((253, 643), (426, 550)), ((385, 328), (406, 194)), ((280, 504), (94, 73)), ((202, 738), (529, 497)), ((7, 240), (97, 467)), ((214, 76), (733, 265)), ((280, 504), (479, 396)), ((169, 619), (179, 257)), ((735, 46), (629, 206)), ((473, 236), (432, 441)), ((750, 376), (639, 343)), ((454, 280), (696, 90)), ((461, 659), (500, 368)), ((621, 155), (575, 329)), ((693, 681), (466, 143)), ((595, 79), (243, 336)), ((211, 20), (36, 336)), ((269, 527), (197, 107)), ((367, 629), (694, 199)), ((597, 328), (658, 694)), ((266, 507), (41, 477)), ((402, 29), (461, 92)), ((299, 568), (639, 343)), ((239, 374), (2, 373)), ((210, 97), (297, 569)), ((745, 27), (257, 149)), ((361, 743), (387, 692)), ((170, 477), (479, 355)), ((339, 353), (224, 248)), ((500, 368), (734, 199)), ((264, 414), (232, 701)), ((578, 99), (417, 158)), ((24, 65), (488, 692)), ((34, 118), (279, 83)), ((400, 513), (79, 333)), ((503, 241), (529, 254)), ((484, 161), (479, 396)), ((178, 519), (649, 57)), ((677, 332), (532, 44)), ((124, 397), (270, 162)), ((267, 352), (155, 193)), ((566, 344), (378, 579)), ((683, 408), (664, 643)), ((266, 244), (269, 527)), ((341, 389), (278, 510)), ((77, 745), (217, 504)), ((403, 218), (403, 218)), ((272, 76), (135, 553)), ((189, 23), (547, 452))]

In [ ]:
flatten_test_points = flatten_one_level(test_points)
flatten_test_points

[(627, 692),
 (170, 274),
 (538, 674),
 (447, 303),
 (581, 20),
 (144, 561),
 (729, 78),
 (331, 367),
 (639, 408),
 (333, 316),
 (254, 504),
 (274, 422),
 (742, 74),
 (299, 568),
 (543, 61),
 (663, 494),
 (380, 155),
 (19, 138),
 (470, 441),
 (623, 47),
 (258, 276),
 (470, 441),
 (717, 601),
 (589, 672),
 (196, 638),
 (529, 497),
 (62, 590),
 (589, 672),
 (69, 21),
 (234, 563),
 (570, 511),
 (30, 515),
 (204, 363),
 (225, 117),
 (285, 96),
 (153, 436),
 (378, 579),
 (272, 76),
 (191, 29),
 (666, 666),
 (30, 515),
 (531, 557),
 (359, 455),
 (566, 344),
 (629, 206),
 (41, 274),
 (196, 638),
 (52, 346),
 (302, 291),
 (54, 620),
 (62, 590),
 (203, 169),
 (320, 577),
 (43, 429),
 (488, 692),
 (39, 402),
 (743, 597),
 (231, 575),
 (233, 495),
 (154, 575),
 (409, 470),
 (487, 380),
 (473, 515),
 (26, 312),
 (32, 451),
 (721, 510),
 (283, 54),
 (508, 668),
 (173, 33),
 (153, 436),
 (200, 220),
 (426, 201),
 (144, 561),
 (676, 558),
 (179, 494),
 (246, 698),
 (480, 495),
 (254, 504),
 (268, 522

In [ ]:
unflatten_test_points = unflatten_one_level(flatten_test_points)
unflatten_test_points

[((627, 692), (170, 274)),
 ((538, 674), (447, 303)),
 ((581, 20), (144, 561)),
 ((729, 78), (331, 367)),
 ((639, 408), (333, 316)),
 ((254, 504), (274, 422)),
 ((742, 74), (299, 568)),
 ((543, 61), (663, 494)),
 ((380, 155), (19, 138)),
 ((470, 441), (623, 47)),
 ((258, 276), (470, 441)),
 ((717, 601), (589, 672)),
 ((196, 638), (529, 497)),
 ((62, 590), (589, 672)),
 ((69, 21), (234, 563)),
 ((570, 511), (30, 515)),
 ((204, 363), (225, 117)),
 ((285, 96), (153, 436)),
 ((378, 579), (272, 76)),
 ((191, 29), (666, 666)),
 ((30, 515), (531, 557)),
 ((359, 455), (566, 344)),
 ((629, 206), (41, 274)),
 ((196, 638), (52, 346)),
 ((302, 291), (54, 620)),
 ((62, 590), (203, 169)),
 ((320, 577), (43, 429)),
 ((488, 692), (39, 402)),
 ((743, 597), (231, 575)),
 ((233, 495), (154, 575)),
 ((409, 470), (487, 380)),
 ((473, 515), (26, 312)),
 ((32, 451), (721, 510)),
 ((283, 54), (508, 668)),
 ((173, 33), (153, 436)),
 ((200, 220), (426, 201)),
 ((144, 561), (676, 558)),
 ((179, 494), (246, 698))

In [ ]:
encoded_str = encode_points_to_unicode(flatten_test_points)
decoded_points = decode_unicode_to_points(encoded_str)

encoded_str, decoded_points

('\U0009ceb4𪤒\U00086aa2\U0006fd2f\U00091414𤈱\U000b644e\U00052d6f\U0009fd98\U0005353c\U0003f9f8\U000449a6\U000b984a\U0004ae38\U00087c3d\U000a5dee\U0005f09b䲊\U000759b9\U0009bc2f\U00040914\U000759b9\U000b3659\U000936a0𱉾\U000845f1祈\U000936a0𑐕\U0003aa33\U0008e9ff稃\U0003316b\U00038475\U00047460𦖴\U0005ea43\U0004404c\U0002fc1d\U000a6a9a稃\U00084e2d\U00059dc7\U0008d958\U0009d4ceꔒ𱉾텚\U0004b923\uda6c祈\U00032ca9\U00050241궭\U0007a2b4鶒\U000b9e55\U00039e3f\U0003a5ef𦨿\U000665d6\U00079d7c\U00076603椸臃\U000b45fe\U00046c36\U0007f29c𫐡𦖴\U000320dc\U0006a8c9𤈱\U000a922e𬷮\U0003daba\U000781ef\U0003f9f8\U0004320a\U0002f6d8\U0006f8db\U000ab989\U0008684d𧫒\U000916db稃\U0005f445㈄\U0006b866\U000ade45䖣\U0008fd49\U0001508a\U0007d17f\U00039e3f\U000b3659\U000b9aa5\U0007aa20\U000158b8𦆞𫣰\U000759b9\U0009b285\U0008d958燁\U00032ae2\U0005353c\U00032405\U000a563c\U00061492\U000871dd\U0009fd57\U0005f09b㈄\U0003e0ec\U0002fed2𤹀\U0008898f\U00054d61\U00062c8d\U00062155鶒\U00079d7c噪\U000af4cb\U000592b8\U00098672酐䲊\U0002f417\U0006c1b9\U0005a

In [ ]:
msgs = "\U0009ceb4𪤒\U00086aa2\U0006fd2f\U00091414𤈱\U000b644e\U00052d6f\U0009fd98\U0005353c\U0003f9f8\U000449a6\U000b984a\U0004ae38\U00087c3d\U000a5dee\U0005f09b䲊\U000759b9\U0009bc2f\U00040914\U000759b9\U000b3659\U000936a0𱉾\U000845f1祈\U000936a0𑐕\U0003aa33\U0008e9ff稃\U0003316b\U00038475\U00047460𦖴\U0005ea43\U0004404c\U0002fc1d\U000a6a9a稃\U00084e2d\U00059dc7\U0008d958\U0009d4ceꔒ𱉾텚\U0004b923祈\U00032ca9\U00050241궭\U0007a2b4鶒\U000b9e55\U00039e3f\U0003a5ef𦨿\U000665d6\U00079d7c\U00076603椸臃\U000b45fe\U00046c36\U0007f29c𫐡𦖴\U000320dc\U0006a8c9𤈱\U000a922e𬷮\U0003daba\U000781ef\U0003f9f8\U0004320a\U0002f6d8\U0006f8db\U000ab989\U0008684d𧫒\U000916db稃\U0005f445㈄\U0006b866\U000ade45䖣\U0008fd49\U0001508a\U0007d17f\U00039e3f\U000b3659\U000b9aa5\U0007aa20\U000158b8𦆞𫣰\U000759b9\U0009b285\U0008d958燁\U00032ae2\U0005353c\U00032405\U000a563c\U00061492\U000871dd\U0009fd57\U0005f09b㈄\U0003e0ec\U0002fed2𤹀\U0008898f\U00054d61\U00062c8d\U00062155鶒\U00079d7c噪\U000af4cb\U000592b8\U00098672酐䲊\U0002f417\U0006c1b9\U0005a6e7\U0004f4b5\U000658c2\U000a606c\U0004b5ed\U00051007\U0006e1c9\U0009d621\U0004e05f\U000512e8\U00050241\U0009bc2f咅\U0009180f\U00084e2d𭡔\U000ae295ॵ𭅗\U000b66a1\U00058a56\U00059a27\U00034edb\U000605a7🧜䲊𫇰\U000975c6\U000a449c\U000429fbꗝ\U0004768f\U00062944\U000a54b3ผ\U0009ea91\U0003d5fd𫃿𡳆\U0009721d\U00095548\U00032405\U000a283a\U000a5d01\U0002fed2\U000430e5\U00060548\U0003daba🤓\U00064201\U0009180f\U00091ae0\U000a6e3b𠇳\U0007b248\U00038ea5\U000aa291\U000428f4𰺶𧚸\U0006fa14\U0009b654\U000b2c53\U000461f8\U000665d6\U0009fd57\U00092683\U000a95a3\U000b0470\U0004eacb\U000196eb\U000535b3\U00065e29\U0009e050\U0009b06a\U00087c3d𬷮\U00051007𬒤\U000aad57\U00058041\U0005cae4\U00065cc6\U00097edd\U0003f8f7\U0004faa0\U000570e4\U0009fd57\U0005be75\U000a6283\U0006b866\U0002fc1d\U000719d7ﰾ乥\U0006e126\U00031684\U00091cb1ॵ\ue5a3\U00076150\U0006481d\U0007345c\U0005cae4\U00074a60\U00040914\U0009924c\U00074a60\U0009488d\U0003364e\U0003b837𘔜\U00055e22\U0004b5ed\U0005d2ca\U000aad98𐨙ᘎ𨐁\U00064656\U0009edfd뚎\U000ae295\U00096136\U000b7ec1\U0003f683\U0006aa26\U00060548\U000658c2\U000461f8𗡉\U00032ae2\U000845f1ᳰ𘗓\U0003584c\U000b7509\U000461f8\U00077d8c𪙫𬴁\U000b7c2e\U0009d4ce\U000764ec\U0006c1b9\U000bb978\U0009fd57\U00071918\U000ae05a\U00073693\U0007d170\U0009b49b\U0008fd49\U000ad6a9\U0007488f\U00094c4f\U0003cd50\U00034c14酐\U0004360f\U0003146b\U0005be75\U000ad8c7\U00095548\U000a4ab6\U000429fbꗝ\U0006481d\U0007345c\U0004ae38\U0009fd57\U0003bd76ॵ\U00034861\U0004a639\U000ba41b\U00040495\U0005a6e7\U00060eb4𪧝\U00077d63\U00054d61\U000380f8\U0007d170\U000b78c7\U0004219e\U0003a2bd\U00090863\U0006849e恁\U0007a2b4衶\U00045c53\U00064201\U00013d4d\U0007dcf1\U000844fe\U000790a1\U00077d8c𬨇\U000a2439\U000a954c\U0008502c🆍\U000438a2\U00042d60𦳁\U0008d958\U0005ea43\U000aad98\U000a6283\U000428f4\U0004360f\U00055585\U000459fe\U000136e9\U000365f8\U00064cda\U00064cda\U0004404c𡸩\U0002f417\U00088dc4"

In [ ]:
decoded_points = decode_unicode_to_points(msgs)

decoded_points

[(627, 692),
 (170, 274),
 (538, 674),
 (447, 303),
 (581, 20),
 (144, 561),
 (729, 78),
 (331, 367),
 (639, 408),
 (333, 316),
 (254, 504),
 (274, 422),
 (742, 74),
 (299, 568),
 (543, 61),
 (663, 494),
 (380, 155),
 (19, 138),
 (470, 441),
 (623, 47),
 (258, 276),
 (470, 441),
 (717, 601),
 (589, 672),
 (196, 638),
 (529, 497),
 (62, 590),
 (589, 672),
 (69, 21),
 (234, 563),
 (570, 511),
 (30, 515),
 (204, 363),
 (225, 117),
 (285, 96),
 (153, 436),
 (378, 579),
 (272, 76),
 (191, 29),
 (666, 666),
 (30, 515),
 (531, 557),
 (359, 455),
 (566, 344),
 (629, 206),
 (41, 274),
 (196, 638),
 (52, 346),
 (302, 291),
 (62, 590),
 (203, 169),
 (320, 577),
 (43, 429),
 (488, 692),
 (39, 402),
 (743, 597),
 (231, 575),
 (233, 495),
 (154, 575),
 (409, 470),
 (487, 380),
 (473, 515),
 (26, 312),
 (32, 451),
 (721, 510),
 (283, 54),
 (508, 668),
 (173, 33),
 (153, 436),
 (200, 220),
 (426, 201),
 (144, 561),
 (676, 558),
 (179, 494),
 (246, 698),
 (480, 495),
 (254, 504),
 (268, 522),
 (189, 72

In [ ]:
for i in range(len(decoded_points)):
  if decoded_points[i] != flatten_test_points[i]:
    print(i, decoded_points[i], flatten_test_points[i])
    break

49 (62, 590) (54, 620)


In [ ]:
# Function to encode (x, y) into two characters from a limited range
def koblitz_encode_to_chars_limited(x, y, field_size_bits=10):
    """
    Encodes the point (x, y) into two characters, ensuring they are from the char_to_num_dict range.
    The range of x and y is limited by the size of field_size_bits.
    """
    # Encode x and y into separate characters
    max_char_index = len(char_to_num_dict) - 1

    # Split x and y into two parts and map them to characters in char_to_num_dict
    char_x = x % len(char_to_num_dict)
    char_y = y % len(char_to_num_dict)

    # Find corresponding characters for x and y
    encoded_char_x = [char for char, num in char_to_num_dict.items() if num == char_x][0]
    encoded_char_y = [char for char, num in char_to_num_dict.items() if num == char_y][0]

    return encoded_char_x, encoded_char_y

# Function to decode from two characters back to (x, y)
def koblitz_decode_from_chars_limited(encoded_char_x, encoded_char_y, field_size_bits=10):
    """
    Decodes two characters back into (x, y) coordinates.
    """
    # Get the corresponding integers from the characters
    x = char_to_num_dict[encoded_char_x]
    y = char_to_num_dict[encoded_char_y]

    return x, y

# Encoding function that uses the limited range and two characters per point
def encode_points_to_unicode_limited(points, field_size_bits=10):
    encoded_chars = []

    for x, y in points:
        encoded_char_x, encoded_char_y = koblitz_encode_to_chars_limited(x, y, field_size_bits)
        encoded_chars.append(encoded_char_x)
        encoded_chars.append(encoded_char_y)

    return ''.join(encoded_chars)

# Decoding function that uses two characters per point
def decode_unicode_to_points_limited(encoded_str, field_size_bits=10):
    decoded_points = []

    for i in range(0, len(encoded_str), 2):
        encoded_char_x = encoded_str[i]
        encoded_char_y = encoded_str[i + 1]
        x, y = koblitz_decode_from_chars_limited(encoded_char_x, encoded_char_y, field_size_bits)
        decoded_points.append((x, y))

    return decoded_points

# Example usage with a list of points
points = flatten_test_points
encoded_str = encode_points_to_unicode_limited(points)
decoded_points = decode_unicode_to_points_limited(encoded_str)

encoded_str, decoded_points


('c.6s5i#g7klsw#3/o/5t8csc4xczak72bwjfjv86cujvkrfgwn#5lgfgsltu#jun-zkz/euq95qzrtaaun/ov4xge10swnbif4d5lg/5x32j#./x5nq1s3v1-j.bmnqpw0oi#dgc9xuq.fg#lskpf201t38cmupv.euo5.zp7yuncscokk/7h9112f8eq1kr4l/b4kt4azjv5uxgs/*05t#55/kn7qofbwco2vrpo2dubpqind/x.bl34/s-zb.8jfpnmvx5uh#u8qe1#7u6ecpd#6x386la8f/oi2-524gfwhqoui6ygd3pjf84v31xkf0q/-p#5f37j1-h89cyu8n0#5zh7brpmog0013tvl8f8/b*570amlo1k/v*y-.-6mi1yc-jofbsl98urij95p*kh/5oakf2#7d0rfoo12*yx.81vgknof/e8skkrt3kmlj/u7xt9d24g9l8xtaa12fycu*efyki0gxef*fde13hr/p45y*1wokh4/-5qn*87sghg0#uyccw*0#57zfg9z.jycsr54fb*5e1mvmvc7of3y-8a38-6w11#pfkl**86k.8nzxp/e*zn02*kf0qxtaaczofy5245fa.7rbqx5i.6qsrbpj28-#zi4r44h7zoo#.y.x1vl*5b.#8x*srerygl4-31so/lowtxg95r/8sk/nzdkwi.7ccydydqzckpne1',
 [(12, 36),
  (6, 28),
  (5, 18),
  (37, 16),
  (7, 20),
  (21, 28),
  (32, 37),
  (3, 39),
  (24, 39),
  (5, 29),
  (8, 12),
  (28, 12),
  (4, 33),
  (12, 35),
  (10, 20),
  (7, 2),
  (11, 32),
  (19, 15),
  (19, 31),
  (8, 6),
  (12, 30),
  (19, 31),
  (20, 27),
  (15, 16),
  (32, 23)

In [ ]:
for i in range(len(decoded_points)):
  if decoded_points[i] != flatten_test_points[i]:
    print(i, decoded_points[i], flatten_test_points[i])
    break

In [ ]:
# Function to encode an integer into a string using characters from char_to_num_dict (base 41)
def encode_number_to_chars(number, base_size):
    encoded_chars = []
    while number > 0:
        remainder = number % base_size
        for char, num in char_to_num_dict.items():
            if num == remainder:
                encoded_chars.append(char)
                break
        number //= base_size
    return ''.join(reversed(encoded_chars)) or '0'  # Handle zero case

# Function to decode a string of characters back into an integer (from base 41)
def decode_chars_to_number(encoded_str, base_size):
    decoded_number = 0
    for char in encoded_str:
        decoded_number = decoded_number * base_size + char_to_num_dict[char]
    return decoded_number

# Function to encode (x, y) into a format where lengths of x and y are preserved
def encode_points_to_unicode_limited(points, field_size_bits=10):
    base_size = len(char_to_num_dict)
    encoded_chars = []

    for x, y in points:
        # Encode x and y into characters
        encoded_x = encode_number_to_chars(x, base_size)
        encoded_y = encode_number_to_chars(y, base_size)

        # Prefix the length of each encoded number for decoding
        length_x = len(encoded_x)
        length_y = len(encoded_y)

        # Add the length of x and y as a prefix (as a single character)
        encoded_chars.append(encode_number_to_chars(length_x, base_size))
        encoded_chars.append(encode_number_to_chars(length_y, base_size))

        # Append encoded x and y
        encoded_chars.append(encoded_x)
        encoded_chars.append(encoded_y)

    return ''.join(encoded_chars)

# Function to decode from the encoded string back to points (x, y)
def decode_unicode_to_points_limited(encoded_str, field_size_bits=10):
    base_size = len(char_to_num_dict)
    decoded_points = []

    i = 0
    while i < len(encoded_str):
        # Decode the lengths of x and y
        length_x = decode_chars_to_number(encoded_str[i], base_size)
        length_y = decode_chars_to_number(encoded_str[i + 1], base_size)
        i += 2

        # Extract x and y based on the lengths
        x_str = encoded_str[i:i + length_x]
        y_str = encoded_str[i + length_x:i + length_x + length_y]
        x = decode_chars_to_number(x_str, base_size)
        y = decode_chars_to_number(y_str, base_size)
        decoded_points.append((x, y))

        i += length_x + length_y  # Move to the next point

    return decoded_points

# Example usage with a list of large points
points = flatten_test_points  # Sample subset of your large list
encoded_str = encode_points_to_unicode_limited(points)
decoded_points = decode_unicode_to_points_limited(encoded_str)

encoded_str, decoded_points


('22fcg.22466s22d5gi22a#7g21e7k223lds22hw1#22838/22fo9/22857t2268cc226sac22i41x227cdz22da1k22g7c2229b3w12j3f22bjav22f816226c6u22bjav22hker22efgg224wfn22c#c5221leg22efgg211sl225tdu22d#cj12ucn224-8z225k2z226/2e223uaq2299e5226q1z214rt22gaga12ucn22c/do228vb422dx8g22fe5122106s224wfn221b8i227f74221df5221leg224/45227xe32212aj22b#g.12/9x22i5en225qe1225sc3223ve1229-bj22b.9b22bmcn12q7p12wb022hoci226#1d22cggc2149x223uaq224.5f22ag4#223lds22gkdp224fc22260h122btc32268cc226mcu224phv22a.5e22gu9o22d51.223zhp22e7hy12ucn229c1s12cco22ak2k22g/e712ha922e18122223f22c89e225qe122hker22i4gl22b/db22244k223ta4224a5z22bjav22f5fu22dx8g12sa/224*i022857t214#522g5d/229k3n22d7bq22fo8f229b3w12cco22625v224rhp223oe222dd9u228b8p229q3i229n8d12/9x22b.9b12lf322h44/228sg-22ezfb12.8812j3f214pn22amav228xi5227u4h229#4u22g82q227ec1217#722aub622fedc227p2d227#i6227xe322f81612l3a21e8f22c/do224i2222g-g512294224g8f22hwgh228qeo228udi2256hy229gad2233bp12j3f2248c422evb322g13x226kcf2210bq226/f-229p7#22g54f123d722fjg1225-ch224869223c4y22eud

In [ ]:
len(encoded_str)

2079

In [ ]:
for i in range(len(decoded_points)):
  if decoded_points[i] != flatten_test_points[i]:
    print(i, decoded_points[i], flatten_test_points[i])
    break